In [3]:
%cd /home/angus_munro/projects/ex3/INF368-Ex3

from keras.models import load_model
from keras.callbacks import CSVLogger
from keras.optimizers import SGD, Adam

import os

from create_model import create_base_network, in_dim, tripletize, std_triplet_loss, alt_triplet_loss
from generators import triplet_generator
import testing as T

import config as C

last = C.last

/home/angus_munro/projects/ex3/INF368-Ex3


It was found that the default base model, which was fully trainable, led to validation loss remaining maxed out at 5.0 during training, at least for 50 epochs. Training loss decreased in a reasonable way. It is likely that the weights in the Inception body were thrown off by the influence of the new last layer weights. To remedy this, weights of the new layer alone were first trained:

In [4]:
from am_plankton import set_trainable_layers

In [5]:
base_model = create_base_network(in_dim)

In [6]:
set_trainable_layers(base_model, 1)
model = tripletize(base_model)
#model.compile(optimizer=SGD(lr=C.learn_rate, momentum=0.9),
#             loss=std_triplet_loss())
model.compile(optimizer=Adam(),
             loss=alt_triplet_loss())

def avg(x):
    return sum(x)/len(x)


Setting top 1 layers of base model trainable


In [40]:
#vs = T.get_vectors(base_model, C.val_dir)

In [5]:
gen = triplet_generator(100, None ,C.val_dir)
#[a,p,n], y = next(gen)
X, y = next(gen)

In [6]:
model.evaluate(X, y, verbose=0) #display the score

4.821909637451172

In [ ]:
logger = CSVLogger(C.logfile, append=True, separator='\t')
model.fit_generator(
        triplet_generator(C.batch_size, None, C.train_dir), steps_per_epoch=300, epochs=C.iterations,
        callbacks=[logger],
        validation_data=triplet_generator(C.batch_size, None, C.val_dir), validation_steps=100)

Epoch 1/5
300/300 [==============================] - 274s 915ms/step - loss: 3.1558 - val_loss: 4.0172
Epoch 2/5
300/300 [==============================] - 241s 802ms/step - loss: 2.7693 - val_loss: 3.8837
Epoch 3/5
281/300 [===========================>..] - ETA: 12s - loss: 2.6571

In [ ]:
model.evaluate(X, y, verbose=0) #display the score

In [ ]:
set_trainable_layers(base_model, 2)
model.compile(optimizer=Adam(),
             loss=alt_triplet_loss())
model.fit_generator(
        triplet_generator(C.batch_size, None, C.train_dir), steps_per_epoch=300, epochs=C.iterations,
        callbacks=[logger],
        validation_data=triplet_generator(C.batch_size, None, C.val_dir), validation_steps=100)

Epoch 1/5
300/300 [==============================] - 220s 734ms/step - loss: 2.5259 - val_loss: 3.8521
Epoch 2/5
158/300 [==============>...............] - ETA: 1:13 - loss: 2.4744

In [28]:
len(model.layers)

5

In [7]:
model.layers

In [8]:
[l.trainable for l in model.layers]

[False, False, False, True, True]

In [9]:
sum(l.trainable for l in base_model.layers)

1

In [10]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
dir(model.layers[-1])

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_built',
 '_compute_elemwise_op_output_shape',
 '_get_node_attribute_at_index',
 '_inbound_nodes',
 '_initial_weights',
 '_losses',
 '_merge_function',
 '_node_key',
 '_non_trainable_weights',
 '_outbound_nodes',
 '_per_input_losses',
 '_per_input_updates',
 '_reshape_required',
 '_trainable_weights',
 '_updates',
 'add_loss',
 'add_update',
 'add_weight',
 'assert_input_compatibility',
 'axis',
 'build',
 'built',
 'call',
 'compute_mask',
 'compute_output_shape',
 'count_params',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'get_input_shape_at',
 'get_losses_for',
 'get_output_at'

In [16]:
model.count_params?

In [23]:
dir(Adam())

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'amsgrad',
 'beta_1',
 'beta_2',
 'decay',
 'epsilon',
 'from_config',
 'get_config',
 'get_gradients',
 'get_updates',
 'get_weights',
 'initial_decay',
 'iterations',
 'lr',
 'set_weights',
 'updates',
 'weights']

In [21]:
for i in range(5 - 1, -1 ,-1):
    print(i)

4
3
2
1
0


In [22]:
'hello{}'.format(7)

'hello7'

In [22]:
%cd /home/angus_munro/projects/ex3/INF368-Ex3
%run train.py

/home/angus_munro/projects/ex3/INF368-Ex3
Setting top 1 layers of base model trainable
Epoch 1/5
300/300 [==============================] - 188s 625ms/step - loss: 3.1652 - val_loss: 2.7403
Epoch 2/5
300/300 [==============================] - 141s 469ms/step - loss: 2.6947 - val_loss: 5.0000
Epoch 3/5
300/300 [==============================] - 141s 469ms/step - loss: 2.6082 - val_loss: 3.1089
Epoch 4/5
300/300 [==============================] - 146s 485ms/step - loss: 2.5392 - val_loss: 5.0000
Epoch 5/5
300/300 [==============================] - 139s 464ms/step - loss: 2.5399 - val_loss: 2.3339
Setting top 9999 layers of base model trainable
Epoch 1/5
300/300 [==============================] - 293s 975ms/step - loss: 2.3516 - val_loss: 5.0000
Epoch 2/5
300/300 [==============================] - 228s 760ms/step - loss: 1.9997 - val_loss: 5.0000
Epoch 3/5
300/300 [==============================] - 228s 760ms/step - loss: 1.7739 - val_loss: 5.0000
Epoch 4/5
300/300 [======================

KeyboardInterrupt: 

It blew up when the whole model waas unlocked. Reduce lr_full to 1E-5 and start from last=1:

In [1]:
%cd /home/angus_munro/projects/ex3/INF368-Ex3
%run train.py

/home/angus_munro/projects/ex3/INF368-Ex3


Using TensorFlow backend.


### Testing triplet_generator ###
0 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
1 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
2 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
3 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Setting top 999 layers of base model trainable
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
600/600 [==============================] - 437s 728ms/step - loss: 2.2110 - val_loss: 1.9027
Epoch 2/5
600/600 [==============================] - 406s 677ms/step - loss: 2.0043 - val_loss: 1.8738
Epoch 3/5
600/600 [==============================] - 405s 675ms/step - loss: 1.8114 - val_loss: 1.9335
Epoch 4/5
600/600 [==============================] - 407s 678ms/step - loss: 1.7061 - val_loss: 1.9462
Epoch 5/5
600/600 [==============================] - 406s 677ms/step - loss: 1.6318 - val_loss: 2.0756
Epoch 1/5
600/600 [==============================] - 418s 697ms/step - loss: 1.5408 - val_loss: 2.1215
Epoch 2/5
600/600 [==============================] - 407s 678ms/step - loss: 1.4859 - val_loss: 2.1888
Epoch 3/5
600/600 [==============================] - 407s 678ms/step - loss: 1.4097 - val_loss: 2.2497
Epoch 4/5
600/600 [==============================] - 407s 679ms/step - loss: 1.37

KeyboardInterrupt: 

In [17]:
model.fit_generator(
        triplet_generator(C.batch_size, None, C.train_dir), 
        steps_per_epoch=C.steps_per_epoch, epochs=C.epochs_per_step,
        callbacks=[logger],
        validation_data=triplet_generator(C.batch_size, None, C.val_dir), validation_steps=100)

Epoch 1/5
300/300 [==============================] - 177s 590ms/step - loss: 3.1165 - val_loss: 3.9523
Epoch 2/5
106/300 [=========>....................] - ETA: 1:18 - loss: 2.7107

KeyboardInterrupt: 

In [18]:
model

In [21]:
dir(model.optimizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'amsgrad',
 'beta_1',
 'beta_2',
 'decay',
 'epsilon',
 'from_config',
 'get_config',
 'get_gradients',
 'get_updates',
 'get_weights',
 'initial_decay',
 'iterations',
 'lr',
 'set_weights',
 'updates',
 'weights']